# Overview

This notebook is the development notes for using `bluesky` and `apstools` to conduct Tomography experiment at 6BM-A at the advanced photon source.

The notebook contains the following sections:
* preparation
    * installation and configuration of `bluesky`+`apstools` for 6BM-A
        * package installation
        * meta-data broker configuration
    * user-define functions that is useful for the tomography experiment
* device declaration: declare `ophyd` interface for `Epics` devices
    * device initilization
* tomography experiment
    * plan generation
    * dry-run
    * connect to devices for experiment

# Preparation

Install with `pip`
------------------

__WARNING__: it is recommended to use conda to install bluesky related pacakge from now.

Install `bluesky` and `apstools` with `pip` (not recommended due to dependency issues):

```bash
pip install -U pip
pip install boltons mongoquery pims pyepics pyRestTable tzlocal jupyter suitcase matplotlib
pip install git+https://github.com/Nikea/historydict#egg=historydict \
            git+https://github.com/NSLS-II/amostra#egg=amostra \
            git+https://github.com/NSLS-II/bluesky#egg=bluesky \
            git+https://github.com/NSLS-II/databroker#egg=databroker \
            git+https://github.com/NSLS-II/doct#egg=doct \
            git+https://github.com/NSLS-II/event-model#egg=event_model \
            git+https://github.com/NSLS-II/ophyd#egg=ophyd \
            git+https://github.com/NSLS-II/hklpy#egg=hklpy
pip install apstools
```

Install with `conda`
--------------------

Install bluesky core packages first
```bash
conda install bluesky -c lightsource2-tag
```

then the apstools dependencies
```bash
conda install pyresttable -c prjemian
```

followed by installing apstools
```bash
conda install apstools -c aps-anl-dev
```

To avoid the warning/runtime-error when running RE, it is recommended to add a file named __pinned__ to the _conda-meta/_ dierectory for given environment with the following content
```shell
tornado<5
```
followed by update the tornado package through conda using 
```bash
conda update tornado
```

> If the command above does not downgrade tornado properly, one can use `conda install tornado==4.5.3` to force reinstall a lower version.

Finally, install jupyter and matplotlib
```bash
conda install jupyter matplotlib
```

Then configure MongoDB for meta-data handling

The following YAML config file should be place under `~/.config/` to use MongoDB as meta-data handler.

```yml
# ~/.config/databroker/mongodb_config.yml

description: 'heavyweight shared database'
metadatastore:
   module: 'databroker.headersource.mongo'
   class: 'MDS'
   config:
       host: 'otz.aps.anl.gov'
       port: 27017
       database: 'metadatastore-production-v1'
       timezone: 'US/Central'
assets:
   module: 'databroker.assets.mongo'
   class: 'Registry'
   config:
       host: 'otz.aps.anl.gov'
       port: 27017
       database: 'filestore-production-v1'
```

In [1]:
# initialize the data base
from databroker import Broker
db = Broker.named("mongodb_config")

# subscribe both mongodb and callback to runtime engine
import bluesky
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback

RE = RunEngine({})
RE.subscribe(db.insert)
RE.subscribe(BestEffortCallback())

/home/beams/S6BM/opt/miniconda3/envs/bluesky_py36/lib/python3.6/site-packages/databroker/_core.py:962: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)


1

In [2]:
import socket
import getpass

socket.gethostname()
getpass.getuser()

's6bm'

In [3]:
# metadata for diagnostics

from datetime import datetime
import apstools
import ophyd
import socket
import getpass

HOSTNAME = socket.gethostname() #or 'localhost'
USERNAME = getpass.getuser() #or '6-BM-A user'
RE.md['beamline_id'] = '6-BM tomo test'
RE.md['login_id'] = USERNAME + '@' + HOSTNAME
RE.md['BLUESKY_VERSION'] = bluesky.__version__
RE.md['OPHYD_VERSION'] = ophyd.__version__
RE.md['apstools_VERSION'] = apstools.__version__
RE.md['SESSION_STARTED'] = datetime.isoformat(datetime.now(), " ")

In [4]:
# user defined function goes here
from pprint import pprint


# 1. checking the APS beam status
import apstools.devices as APS_devices

aps = APS_devices.ApsMachineParametersDevice(name="APS")

pprint(aps)

ApsMachineParametersDevice(prefix='', name='APS', read_attrs=['current', 'lifetime', 'machine_status', 'operating_mode', 'shutter_permit', 'fill_number', 'orbit_correction', 'global_feedback', 'global_feedback_h', 'global_feedback_v', 'operator_messages', 'operator_messages.operators', 'operator_messages.floor_coordinator', 'operator_messages.fill_pattern', 'operator_messages.last_problem_message', 'operator_messages.last_trip_message', 'operator_messages.message6', 'operator_messages.message7', 'operator_messages.message8'], configuration_attrs=['operator_messages'])


Now from the hardware end, we need additional setup IOC (soft) for this status, recommended method `VME`

Here is a quick example from 2BM
```python
instrument_in_use = EpicsSignalRO(
    "2bm:instrument_in_use", 
    name="instrument_in_use")

def operations_in_2bmb():
    """returns True if allowed to use X-ray beam in 2-BM-B station"""
    v = instrument_in_use.value
    enums = instrument_in_use.enum_strs
    return enums[v] == "2-BM-B"
```

In [5]:
# 2. assume we have the same setup describe above

from ophyd import EpicsSignalRO

instrument_in_use = EpicsSignalRO("6bm:instrument_in_use", name="instrument_in_use")

RE.md['INSTRUMENT_IN_USE'] = instrument_in_use.value

In [6]:
print(instrument_in_use)

EpicsSignalRO(read_pv='6bm:instrument_in_use', name='instrument_in_use', value=1, timestamp=1554729382.179483, auto_monitor=False, string=False)


In [7]:
# 3. check ambien light in hutch
import apstools.synApps_ophyd

# grab all calcs
calcs = apstools.synApps_ophyd.userCalcsDevice("6bma1:", name="calcs")

# calc1.ch8 is ambient light checker
#calc1 = calcs.calc1
hutch_light_on = bool(calcs.calc1.val.value)

In [8]:
print(calcs.calc1.val.value ,hutch_light_on)

1.0 True


In [9]:
apstools.synApps_ophyd.userCalcsDevice??

In [10]:
# 4. define the ENV VAR denote the experiment nature
# NOTE:
#    the return value of instrument_in_use is not yet decided

# conducting experiment mode
in_production = aps.inUserOperations \
            and (instrument_in_use.value in (1, "6-BM-A")) \
            and (not hutch_light_on)

# testing mode, supercede in_production
in_dryrun = True

In [11]:
print(in_production, aps.inUserOperations)

False False


# Device declaration

In [12]:
# necessary import
import apstools.devices as APS_devices

## shutter

In [13]:

if in_production:
    # define the real shutter used at 6BMA@APS
    # NOTE: 
    #   this requires connection to the hardware, otherwise a connection error will be raised

    A_shutter = APS_devices.ApsPssShutterWithStatus(
            "6bmb1:rShtrA",
            "PA:06BM:STA_A_FES_OPEN_PL",
            name="A_shutter",
        )
    
    # no scans until A_shutter is open
    suspend_A_shutter = bluesky.suspenders.SuspendFloor(A_shutter.pss_state, 1)
    #suspend_A_shutter.install(RE)
    RE.install_suspender(suspend_A_shutter)
    
    # no scans if aps.current is too low
    suspend_APS_current = bluesky.suspenders.SuspendFloor(aps.current, 2, resume_thresh=10)
    RE.install_suspender(suspend_APS_current)

else:
    # first, a simulated shutter to demonstrate the design of epics controled shutter

    A_shutter = APS_devices.SimulatedApsPssShutterWithStatus(name="A_shutter")

    A_shutter.summary()

data keys (* hints)
-------------------
 A_shutter_busy
 A_shutter_close_signal
 A_shutter_open_signal
 A_shutter_pss_state

read attrs
----------
busy                 Signal              ('A_shutter_busy')
open_signal          Signal              ('A_shutter_open_signal')
close_signal         Signal              ('A_shutter_close_signal')
pss_state            Signal              ('A_shutter_pss_state')

config keys
-----------

configuration attrs
-------------------

unused attrs
------------



In [14]:
A_shutter.isOpen

False

To open a shutter, use `A_shutter.open`. 
Similarly, the shutter can be closed with `A_shutter.close`.
One can also check the current status of the shutter using `A_shutter.isOpen`.

## area detector

In [77]:
import os
import datetime
from pathlib import Path, PureWindowsPath

# production control ENV vars
ADPV_prefix = "1idPG2"   # AreaDetector prefix
# OUTPUT_ROOT = "/home/beams/S6BM/user_data"
OUTPUT_ROOT = "Y:\\"

# -----------------------
# -- user config block --
# -----------------------
CYCLE = "2019-1"
EXPID = "internal_apr19"
USER  = EXPID
SAMPLE = "tester"

FILE_PATH = str(PureWindowsPath(Path("/".join([OUTPUT_ROOT, CYCLE, EXPID, "tomo", SAMPLE])+"/")))
# FILE_PATH =  os.path.join(OUTPUT_ROOT, CYCLE, EXPID, "tomo", SAMPLE) + os
FILE_PREFIX = SAMPLE

# show where files will be stored
print(FILE_PATH)
print(FILE_PREFIX)

Y:\2019-1\internal_apr19\tomo\tester
tester


In [78]:
# As of today, there is still no support for Python class enclosure, so we have to make a class for the detector
from ophyd import AreaDetector
from ophyd import SingleTrigger
from ophyd import ADComponent
from ophyd import PointGreyDetectorCam
from ophyd import ProcessPlugin
from ophyd import TIFFPlugin
from ophyd import sim

# class PointGreyDetectorCam6BM(PointGreyDetectorCam):
#     """Extend the default Point Grey detector cam"""
#     frame_rate_on_off = ADComponent(EpicsSignalWithRBV, "FrameRateOnOff")


class PointGreyDetector6BM(SingleTrigger, AreaDetector):
    """Point Gray area detector used at 6BM"""
    # cam component
    cam = ADComponent(PointGreyDetectorCam, "cam1:")
    
    # proc plugin
    proc1 = ADComponent(ProcessPlugin, suffix="Proc1:")
    
    # tiff plugin
    tiff1 = ADComponent(
        TIFFPlugin,
        suffix="TIFF1:",
#         root=OUTPUT_ROOT,                       # for databroker
#         write_path_template=FILE_PATH,          # for EPICS AD
    )
    

# ------------------------------------
# ----- Instantiate the detector ----- 
# ------------------------------------

# Area Detector (AD) config block
config_cam = {
    "num_images":     1,           # number of images (nFrame)
    "image_mode":     "Multiple",  #
    "trigger_mode":   "Internal",  #
    "acquire_time":   0.05,           # exposure time (fExposureTime)
    "acquire_period": 0.05+0.01,      #
    "gain":           5,           # detector gain [0~30]
}

config_proc1 = {
    "enable":           1,  # toggle on proc1
    "enable_filter":    1,  # enable filter
    "num_filter":       5,  # change number_filtered in proc1 (same as nFrame)
    "reset_filter":     1,  # reset number_filtered
}

config_tiff1 = {
    "nd_array_port":    "PROC1",      # switch port for TIFF plugin
    "file_write_mode":  "Stream",     # change write mode
    "auto_save":        "Yes",        # turn on file save
    "file_path":        FILE_PATH,    # set file path
    "file_name":        FILE_PREFIX,  # img name prefix
}


if in_production or in_dryrun:
    pg2_det = PointGreyDetector6BM("1idPG2:", name='pg2_det')
    
    # catch timeout error in case detector not responding
    try:
        for k, v in config_cam.items():     pg2_det.cam.stage_sigs[k]   = v
        for k, v in config_proc1.items():   pg2_det.proc1.stage_sigs[k] = v
        for k, v in config_tiff1.items():   pg2_det.tiff1.stage_sigs[k] = v
    except TimeoutError as _exc:
        print(f"{_exc}\n !! Could not connect with area detector {pg2_det}")
else:
    pg2_det = sim.noisy_det  # use ophyd simulated detector

In [79]:
pg2_det.tiff1.file_path.put(FILE_PATH)

## motor

In [19]:
from ophyd import MotorBundle
from ophyd import Component
from ophyd import EpicsMotor

# NOTE: 
#    the PV for actual motors is still unknown
class TomoStage(MotorBundle):
    #rotation
    preci = Component(EpicsMotor, "6bmpreci:m1",
                      name='preci',
                     )
    
    samX = Component(EpicsMotor, "6bma1:m19",
                     name='samX',
                    )
    
    samY = Component(EpicsMotor, "6bma1:m18",
                     name="samY",
                    )

if in_production or in_dryrun:
    tomostage = TomoStage(name='tomostage')

    samx  = tomostage.samX
    samy  = tomostage.samY
    preci = tomostage.preci

else:
    tomostage = MotorBundle()
    tomostage.preci = sim.motor
    tomostage.samX = sim.motor
    tomostage.samY = sim.motor


In [ ]:
preci.move(0)

# Tomography experiment

## Scan plans

Define the scan parameters within plan functions

In [22]:
import bluesky.plans as bp
import bluesky.plan_stubs as bps
import numpy as np

# ---------------------------
# ----- Scan parameters -----
# ---------------------------
n_white    = 10
n_dark     = 10
samOutDist = -2.00                           # mm
omega_step = 0.25                            # degrees
time_wait  = 1                            # s
omega_start = 0
omega_end   = 5 
number_of_projections = int(abs(omega_end-omega_start)/omega_step)+1  # 0 ~ 180


def check_images_n():
    return True
    while (pg2_det.tiff1.num_captured.value < n_white): 
        yield from bps.sleep(time_wait)
        
    
def collect_white_field():
    """collect 10 white field images before and after tomo scan"""
    yield from bps.mv(
        A_shutter,              "open", 
#         pg2_det.cam.trigger_mode,   "Internal",
#         pg2_det.cam.num_images,     n_white,
#         pg2_det.proc1.reset_filter, 1,
    )
    
    yield from bps.trigger(pg2_det)
    
    yield from check_images_n()


def collect_dark_field():
    """collect 10 dark field images after tomo scan"""
    
    yield from bps.mv(
        A_shutter,              "close", 
#         pg2_det.cam.trigger_mode,   "Internal",
#         pg2_det.cam.num_images,     n_dark,
#         pg2_det.proc1.reset_filter, 1,
    )
    
    yield from bps.trigger(pg2_det)
    
    yield from check_images_n()
    
    
def collect_projection():
    """tomo scna"""
    yield from bp.scan(
        [pg2_det], 
        preci, omega_start, omega_end, number_of_projections,
        per_step=bps.sleep(time_wait),
    )
    
def tomo_scan():
    # white field images
    current_samx = samx.position
    yield from bps.mv( samx, current_samx+samOutDist)
    yield from collect_white_field()
    yield from bps.mv( samx, current_samx)
    
    # proections
    # NOTE:
    #   use bps.sleep to semi-pause scan
    yield from collect_projection()
#     for i in np.arange(omega_start, omega_end, omega_step):
#         yield from bp.scan( [pg2_det], preci, i, i+omega_step, 1)
#         yield from bps.sleep(time_wait)
    
    # white field images
    yield from bps.mv( samx, current_samx-samOutDist)
    yield from collect_white_field()
    yield from bps.mv( samx, current_samx)
    
    # dark field images
    yield from collect_dark_field()


## Scan plan pruning prior to execution

Use the `summarize_plan` feature from bluesky to perform sanity check on the tomo scan plan

In [24]:
from bluesky.simulators import summarize_plan

summarize_plan(tomo_scan())

tomostage_samX -> -2.0
A_shutter -> open
tomostage_samX -> 0.0
=================================== Open Run ===================================
tomostage_preci -> 0.0
  Read ['noisy_det', 'tomostage_preci']
================================== Close Run ===================================
=================================== Open Run ===================================
tomostage_preci -> 0.25
  Read ['noisy_det', 'tomostage_preci']
================================== Close Run ===================================
=================================== Open Run ===================================
tomostage_preci -> 0.5
  Read ['noisy_det', 'tomostage_preci']
================================== Close Run ===================================
=================================== Open Run ===================================
tomostage_preci -> 0.75
  Read ['noisy_det', 'tomostage_preci']
================================== Close Run ===================================
===================================

## Excuete plans

In [25]:
RE(tomo_scan())

RuntimeError: This event loop is already running

Transient Scan ID: 1     Time: 2019-04-08 15:55:50
Persistent Unique Scan ID: 'eaacdda6-b097-48ff-8ed0-67be3148415f'
New stream: 'primary'
+-----------+------------+-----------------+
|   seq_num |       time | tomostage_preci |
+-----------+------------+-----------------+
|         1 | 15:55:52.0 |      0.00015256 |
+-----------+------------+-----------------+
generator scan ['eaacdda6'] (scan num: 1)



Transient Scan ID: 2     Time: 2019-04-08 15:55:53
Persistent Unique Scan ID: '638a6adf-b0de-4338-ac1b-9164ba0b18fb'
New stream: 'primary'
+-----------+------------+-----------------+
|   seq_num |       time | tomostage_preci |
+-----------+------------+-----------------+
|         1 | 15:55:53.6 |      0.24951188 |
+-----------+------------+-----------------+
generator scan ['638a6adf'] (scan num: 2)



Transient Scan ID: 3     Time: 2019-04-08 15:55:54
Persistent Unique Scan ID: '1f3dcf8b-35fc-41ee-839c-dcca1531b9cc'
New stream: 'primary'
+-----------+------------+-----------------

ERROR:root:Invalid alias: The name clear can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name more can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name less can't be aliased because it is another magic command.
ERROR:root:Invalid alias: The name man can't be aliased because it is another magic command.


In [1]:
import bluesky.plans as bp

In [3]:
bp.scan??